# Linked List Intersection

* Return the node where 2 singly linked lists intersect (none otherwise)
* The intersection has nothing to do with the values of the nodes
   
**Complexity Analysis :**

One pass


| Time | Space |
|------|-------|
| O(n+m) | O(1)|

- Time, because we traverse both lists A (of length n) and B (of length m) 
- Space, because in place


## Build'n print lists

In [3]:
use std::rc::Rc;
type Link = Option<Rc<Node>>; // type alias. Use Option and Rc to allow an optional reference count to the next node 

struct Node {
    val: i32,
    next: Link, // use the alias here for clarity
}

impl Node {
    fn new(val: i32, next: Link) -> Node {
        Node { val, next }
    }
}


// fn main(){     // no main() if this code runs in a jupyter cell 
{                 // local scope to avoid issue with the lifetime of head during borrow

    let mut head_common = None; // Start with an empty list (head is None)
    let vals_common = vec![4, 8, 7, 2];
    for v in vals_common.into_iter().rev() {
        head_common = Some(Rc::new(Node::new(v, head_common)));
    }

    // This is NOT a copy. Here .clone() add a new pointer to head_common
    let mut head_A = head_common.clone(); 
    let vals_A = vec![1, 3];
    for v in vals_A.into_iter().rev() {
        head_A = Some(Rc::new(Node::new(v, head_A)));
    }

    let mut current = &head_A;
    while let Some(node) = current {
        print!("{} -> ", node.val);
        current = &node.next;
    }
    println!("None");


    let mut head_B = head_common.clone(); 
    let vals_B = vec![6];
    for v in vals_B.into_iter().rev() {
        head_B = Some(Rc::new(Node::new(v, head_B)));
    }

    let mut current = &head_B;
    while let Some(node) = current {
        print!("{} -> ", node.val);
        current = &node.next;
    }
    println!("None");

} // end of local scope OR end of main()   



1 -> 3 -> 4 -> 8 -> 7 -> 2 -> None
6 -> 4 -> 8 -> 7 -> 2 -> None


()

In [4]:
use std::rc::Rc;
type Link = Option<Rc<Node>>;

struct Node {
    val: i32,
    next: Link, // use the alias here for clarity
}

impl Node {
    fn new(val: i32, next: Link) -> Node {
        Node { val, next }
    }
}

fn print_linked_list(head: &Link, name: &str) {
    print!("{} : ", name);
    let mut current = head;
    while let Some(node) = current {
        print!("{} -> ", node.val);
        current = &node.next;
    }
    println!("None");
}

fn linked_list_intersection(head_A: &Link, head_B: &Link) -> Link {
    let mut ptr_A = head_A;
    let mut ptr_b = head_B;

    loop {
        match (&ptr_A, &ptr_b) {
            (Some(a), Some(b)) => {
                if Rc::ptr_eq(a, b) {
                    // .clone() is mandatory here
                    // It is called on the referenced value (Option<Rc<Node>>) NOT on the the reference itself (&Option<Rc<Node>>)
                    // None or Some(Rc<Node>) is cloned (cheap operation since the reference counter is incremented)
                    return ptr_A.clone();
                }
            }
            (None, None) => return None,
            _ => {}
        }
        // If we reach the end of A, we start traversing B
        ptr_A = match ptr_A {
            Some(node) => &node.next,
            None => head_B,
        };
        // Traverse B then A
        ptr_b = match ptr_b {
            Some(node) => &node.next,
            None => head_A,
        };
    }
}


// fn main(){     // no main() if this code runs in a jupyter cell 
{                 // local scope to avoid issue with the lifetime of head during borrow

    let mut head_common = None; // Start with an empty list (head is None)
    let vals_common = vec![8, 7, 2];
    for v in vals_common.into_iter().rev() {
        head_common = Some(Rc::new(Node::new(v, head_common)));
    }
    // print_linked_list(&head_common, "List Common");

    // This is NOT a copy. Here .clone() add a new pointer to head_common
    let mut head_A = head_common.clone(); 
    let vals_A = vec![1, 3, 4];
    for v in vals_A.into_iter().rev() {
        head_A = Some(Rc::new(Node::new(v, head_A)));
    }
    print_linked_list(&head_A, "List A");


    let mut head_B = head_common.clone(); 
    let vals_B = vec![6, 5];
    for v in vals_B.into_iter().rev() {
        head_B = Some(Rc::new(Node::new(v, head_B)));
    }
    print_linked_list(&head_B, "List B");

    let intersection = linked_list_intersection(&head_A, &head_B);
    print_linked_list(&intersection, "Intersection");

} // end of local scope OR end of main()       

List A : 1 -> 3 -> 4 -> 8 -> 7 -> 2 -> None
List B : 6 -> 5 -> 8 -> 7 -> 2 -> None
Intersection : 8 -> 7 -> 2 -> None


()